In [ ]:
import pandas as pd  

# 读取CSV文件  
file_path = 'RESSET_DRESSTK_2021__1.csv'  # 替换为你的CSV文件路径  
df = pd.read_csv(file_path)  

# 删除包含缺失数据的行  
df_clean = df.dropna()  

# 将清理后的数据保存到一个新的CSV文件  
output_file_path = 'cleaned_file.csv'  # 替换为你想要保存的文件路径  
df_clean.to_csv(output_file_path, index=False)  

print("清理完成，缺失数据的行已被删除。")

In [ ]:
filepath = 'cleaned_file.csv'
data = pd.read_csv(filepath)
data = data.sort_values('Date')
data.head()

In [ ]:
price = data[['Clpr', 'Trdvol', 'Trdsum', 'Dret', 'PE', 'PB', 'PCF', 'PS', 'EPS', 'ROE']]
price


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scalerClpr = MinMaxScaler(feature_range=(-1, 1))
scalerTrdvol = MinMaxScaler(feature_range=(-1, 1))
scalerTrdsum = MinMaxScaler(feature_range=(-1, 1))
scalerDret = MinMaxScaler(feature_range=(-1, 1))
scalerPE = MinMaxScaler(feature_range=(-1, 1))
scalerPB = MinMaxScaler(feature_range=(-1, 1))
scalerPCF = MinMaxScaler(feature_range=(-1, 1))
scalerPS = MinMaxScaler(feature_range=(-1, 1))
scalerEPS = MinMaxScaler(feature_range=(-1, 1))
scalerROE = MinMaxScaler(feature_range=(-1, 1))
price['Clpr'] = scalerClpr.fit_transform(price['Clpr'].values.reshape(-1,1))
price['Trdvol'] = scalerTrdvol.fit_transform(price['Trdvol'].values.reshape(-1,1))
price['Trdsum'] = scalerTrdsum.fit_transform(price['Trdsum'].values.reshape(-1,1))
price['Dret'] = scalerDret.fit_transform(price['Dret'].values.reshape(-1,1))
price['PE'] = scalerPE.fit_transform(price['PE'].values.reshape(-1,1))
price['PB'] = scalerPB.fit_transform(price['PB'].values.reshape(-1,1))
price['PCF'] = scalerPCF.fit_transform(price['PCF'].values.reshape(-1,1))
price['PS'] = scalerPS.fit_transform(price['PS'].values.reshape(-1,1))
price['EPS'] = scalerEPS.fit_transform(price['EPS'].values.reshape(-1,1))
price['ROE'] = scalerROE.fit_transform(price['ROE'].values.reshape(-1,1))
price.head()
print(price)

# 创建一个空的列表来存储新的 DataFrame 对象  
new_dataframes = []  
  
# 循环创建九个新的 DataFrame  
for shift in range(9):  
    # 计算新的列顺序  
    new_columns = [price.columns[(shift + 1 + idx) % len(price.columns)] for idx in range(len(price.columns))]  
    # 通过重新排序列来创建新的 DataFrame  
    # 注意：这里我们实际上没有移动数据，只是改变了列的显示顺序  
    new_df = price[new_columns]  
    # 将新的 DataFrame 添加到列表中  
    new_dataframes.append(new_df) 
    
new_dataframes[0].head()
    



In [ ]:
#数据集
import numpy as np
def split_data(stock, lookback):
    data_raw = stock.to_numpy() 
    print('data_raw.shape = ',data_raw.shape)
    data = []
    
    # you can free play（seq_length）
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    
    data = np.array(data);
    print('data.shape = ',data.shape)
    test_set_size = int(np.round(0.2 * data.shape[0]))
    train_set_size = data.shape[0] - (test_set_size)
    
    x_train = data[:train_set_size,:-1,:]
    #print(x_train)
    y_train = data[:train_set_size,-1,0:1]
    
    x_test = data[train_set_size:,:-1,:]
    y_test = data[train_set_size:,-1,0:1]
    
    return [x_train, y_train, x_test, y_test]

In [ ]:
lookback = 20
x_train0=[]
y_train0=[]
x_test0=[]
y_test0=[]
x_train1, y_train1, x_test1, y_test1 = split_data(price, lookback)
x_train0.append(x_train1)
y_train0.append(y_train1)
x_test0.append(x_test1)
y_test0.append(y_test1)
for i in range(9):
    xt, yt, xt1, yt1 = split_data(new_dataframes[i], lookback)
    x_train0.append(xt)
    y_train0.append(yt)
    x_test0.append(xt1)
    y_test0.append(yt1)
print('x_train.shape = ',len(x_train0))
#print(x_train)
print('y_train.shape = ',len(y_train0))
print('x_test.shape = ',len(x_test0))
print('y_test.shape = ',len(y_test0))
print(x_test0[0].shape)

In [ ]:
import torch
import torch.nn as nn

x_train=[]
x_test=[]
y_train_lstm=[]
y_test_lstm=[]
y_train_gru=[]
y_test_gru=[]


for i in range(10):
    x_trainn = torch.from_numpy(x_train0[i]).type(torch.Tensor)
    x_testn = torch.from_numpy(x_test0[i]).type(torch.Tensor)
    y_train_lstmn = torch.from_numpy(y_train0[i]).type(torch.Tensor)
    y_test_lstmn = torch.from_numpy(y_test0[i]).type(torch.Tensor)
    y_train_grun = torch.from_numpy(y_train0[i]).type(torch.Tensor)
    y_test_grun = torch.from_numpy(y_test0[i]).type(torch.Tensor)
    x_train.append(x_trainn)
    x_test.append(x_testn)
    y_train_lstm.append(y_train_lstmn)
    y_test_lstm.append(y_test_lstmn)
    y_train_gru.append(y_train_grun)
    y_test_gru.append(y_test_grun)
    

In [ ]:
input_dim = 10
hidden_dim = 32
num_layers = 2
output_dim = 1
num_epochs = 500

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [ ]:
model0 = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
model1 = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
model2 = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
model3 = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
model4 = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
model5 = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
model6 = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
model7 = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
model8 = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
model9 = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss()
optimiser0 = torch.optim.Adam(model0.parameters(), lr=0.01)
optimiser1 = torch.optim.Adam(model1.parameters(), lr=0.01)
optimiser2 = torch.optim.Adam(model2.parameters(), lr=0.01)
optimiser3 = torch.optim.Adam(model3.parameters(), lr=0.01)
optimiser4 = torch.optim.Adam(model4.parameters(), lr=0.01)
optimiser5 = torch.optim.Adam(model5.parameters(), lr=0.01)
optimiser6 = torch.optim.Adam(model6.parameters(), lr=0.01)
optimiser7 = torch.optim.Adam(model7.parameters(), lr=0.01)
optimiser8 = torch.optim.Adam(model8.parameters(), lr=0.01)
optimiser9 = torch.optim.Adam(model9.parameters(), lr=0.01)

In [ ]:
#模型训练
import time

hist0 = np.zeros(num_epochs)
start_time = time.time()
lstm = []

for t in range(num_epochs):
    y_train_pred0 = model0(x_train[0])

    loss = criterion(y_train_pred0, y_train_lstm[0])
    print("Epoch ", t, "MSE: ", loss.item())
    hist0[t] = loss.item()

    optimiser0.zero_grad()
    loss.backward()
    optimiser0.step()
    
for t in range(num_epochs):
    y_train_pred1=model1(x_train[1])

    loss = criterion(y_train_pred1, y_train_lstm[1])
    print("Epoch ", t, "MSE: ", loss.item())
    #hist[t] = loss.item()

    optimiser1.zero_grad()
    loss.backward()
    optimiser1.step()

for t in range(num_epochs):
    y_train_pred2=model2(x_train[2])

    loss = criterion(y_train_pred2, y_train_lstm[2])
    print("Epoch ", t, "MSE: ", loss.item())
    #hist[t] = loss.item()

    optimiser2.zero_grad()
    loss.backward()
    optimiser2.step()

for t in range(num_epochs):
    y_train_pred3=model3(x_train[3])

    loss = criterion(y_train_pred3, y_train_lstm[3])
    print("Epoch ", t, "MSE: ", loss.item())
    #hist[t] = loss.item()

    optimiser3.zero_grad()
    loss.backward()
    optimiser3.step()
    
for t in range(num_epochs):
    y_train_pred4=model4(x_train[4])

    loss = criterion(y_train_pred4, y_train_lstm[4])
    print("Epoch ", t, "MSE: ", loss.item())
    #hist[t] = loss.item()

    optimiser4.zero_grad()
    loss.backward()
    optimiser4.step()
    
for t in range(num_epochs):
    y_train_pred5=model5(x_train[5])

    loss = criterion(y_train_pred5, y_train_lstm[5])
    print("Epoch ", t, "MSE: ", loss.item())
    #hist[t] = loss.item()

    optimiser5.zero_grad()
    loss.backward()
    optimiser5.step()
    
for t in range(num_epochs):
    y_train_pred6=model6(x_train[6])

    loss = criterion(y_train_pred6, y_train_lstm[6])
    print("Epoch ", t, "MSE: ", loss.item())
    #hist[t] = loss.item()

    optimiser6.zero_grad()
    loss.backward()
    optimiser6.step()
    
for t in range(num_epochs):
    y_train_pred7=model7(x_train[7])

    loss = criterion(y_train_pred7, y_train_lstm[7])
    print("Epoch ", t, "MSE: ", loss.item())
    #hist[t] = loss.item()

    optimiser7.zero_grad()
    loss.backward()
    optimiser7.step()
    
for t in range(num_epochs):
    y_train_pred8=model8(x_train[8])

    loss = criterion(y_train_pred8, y_train_lstm[8])
    print("Epoch ", t, "MSE: ", loss.item())
    #hist[t] = loss.item()

    optimiser8.zero_grad()
    loss.backward()
    optimiser8.step()

for t in range(num_epochs):
    y_train_pred9=model9(x_train[9])

    loss = criterion(y_train_pred9, y_train_lstm[9])
    print("Epoch ", t, "MSE: ", loss.item())
    #hist[t] = loss.item()

    optimiser9.zero_grad()
    loss.backward()
    optimiser9.step()

In [ ]:
predict = pd.DataFrame(scalerClpr.inverse_transform(y_train_pred1.detach().numpy()))
original = pd.DataFrame(scalerClpr.inverse_transform(y_train_lstm[1].detach().numpy()))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("darkgrid")    

fig = plt.figure()
fig.subplots_adjust(hspace=0.2, wspace=0.2)

plt.subplot(1, 2, 1)
ax = sns.lineplot(x = original.index, y = original[0], label="Data", color='royalblue')
ax = sns.lineplot(x = predict.index, y = predict[0], label="Training Prediction (LSTM)", color='tomato')
ax.set_title('Stock price', size = 14, fontweight='bold')
ax.set_xlabel("Days", size = 14)
ax.set_ylabel("Cost (USD)", size = 14)
ax.set_xticklabels('', size=10)


plt.subplot(1, 2, 2)
ax = sns.lineplot(data=hist0, color='royalblue')
ax.set_xlabel("Epoch", size = 14)
ax.set_ylabel("Loss", size = 14)
ax.set_title("Training Loss", size = 14, fontweight='bold')
fig.set_figheight(6)
fig.set_figwidth(16)

In [ ]:

""" #test验证
 import math, time
from sklearn.metrics import mean_squared_error
 
# make predictions
y_test_pred = model(x_test)

# invert predictions
y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
y_train = scaler.inverse_transform(y_train_lstm.detach().numpy())
y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
y_test = scaler.inverse_transform(y_test_lstm.detach().numpy())

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train[:,0], y_train_pred[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test[:,0], y_test_pred[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
lstm.append(trainScore)
lstm.append(testScore)
lstm.append(training_time) """

In [ ]:
""" # 注意这里标签索引是一个列表
new_price = price[['Clpr']]
new_price """

In [ ]:
""" # shift train predictions for plotting
trainPredictPlot = np.empty_like(new_price)
trainPredictPlot[:, 0] = np.nan
trainPredictPlot[lookback:len(y_train_pred)+lookback, :] = y_train_pred """

In [ ]:
""" # shift test predictions for plotting
testPredictPlot = np.empty_like(new_price)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(y_train_pred)+lookback-1:len(price)-1, :] = y_test_pred """

In [ ]:
""" original = scaler.inverse_transform(price['Clpr'].values.reshape(-1,1))

predictions = np.append(trainPredictPlot, testPredictPlot, axis=1)
predictions = np.append(predictions, original, axis=1)
result = pd.DataFrame(predictions) """

In [ ]:
""" import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(go.Scatter(x=result.index, y=result[0],
                    mode='lines',
                    name='Train prediction')))
fig.add_trace(go.Scatter(x=result.index, y=result[1],
                    mode='lines',
                    name='Test prediction'))
fig.add_trace(go.Scatter(go.Scatter(x=result.index, y=result[2],
                    mode='lines',
                    name='Actual Value')))
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=False,
        linecolor='white',
        linewidth=2
    ),
    yaxis=dict(
        title_text='Close (USD)',
        titlefont=dict(
            family='Rockwell',
            size=12,
            color='white',
        ),
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='white',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Rockwell',
            size=12,
            color='white',
        ),
    ),
    showlegend=True,
    template = 'plotly_dark'

)



annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Results (LSTM)',
                              font=dict(family='Rockwell',
                                        size=26,
                                        color='white'),
                              showarrow=False))
fig.update_layout(annotations=annotations)

fig.show() """

In [ ]:
filepath = 'RESSET_DRESSTK_2021.csv'
data2 = pd.read_csv(filepath)
data2 = data2.sort_values('Date')
data2.head()

In [ ]:
price2 = data2[['Clpr', 'Trdvol', 'Trdsum', 'Dret', 'PE', 'PB', 'PCF', 'PS', 'EPS', 'ROE']]
price2

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
scalerClpr = MinMaxScaler(feature_range=(-1, 1))
scalerTrdvol = MinMaxScaler(feature_range=(-1, 1))
scalerTrdsum = MinMaxScaler(feature_range=(-1, 1))
scalerDret = MinMaxScaler(feature_range=(-1, 1))
scalerPE = MinMaxScaler(feature_range=(-1, 1))
scalerPB = MinMaxScaler(feature_range=(-1, 1))
scalerPCF = MinMaxScaler(feature_range=(-1, 1))
scalerPS = MinMaxScaler(feature_range=(-1, 1))
scalerEPS = MinMaxScaler(feature_range=(-1, 1))
scalerROE = MinMaxScaler(feature_range=(-1, 1))
price2['Clpr'] = scalerClpr.fit_transform(price2['Clpr'].values.reshape(-1,1))
price2['Trdvol'] = scalerTrdvol.fit_transform(price2['Trdvol'].values.reshape(-1,1))
price2['Trdsum'] = scalerTrdsum.fit_transform(price2['Trdsum'].values.reshape(-1,1))
price2['Dret'] = scalerDret.fit_transform(price2['Dret'].values.reshape(-1,1))
price2['PE'] = scalerPE.fit_transform(price2['PE'].values.reshape(-1,1))
price2['PB'] = scalerPB.fit_transform(price2['PB'].values.reshape(-1,1))
price2['PCF'] = scalerPCF.fit_transform(price2['PCF'].values.reshape(-1,1))
price2['PS'] = scalerPS.fit_transform(price2['PS'].values.reshape(-1,1))
price2['EPS'] = scalerEPS.fit_transform(price2['EPS'].values.reshape(-1,1))
price2['ROE'] = scalerROE.fit_transform(price2['ROE'].values.reshape(-1,1))
price2.head()
print(price2)

# 创建一个空的列表来存储新的 DataFrame 对象  
new_dataframes2 = []  

# 循环创建九个新的 DataFrame  
for shift in range(9):  
    # 计算新的列顺序  
    new_columns = [price2.columns[(shift + 1 + idx) % len(price2.columns)] for idx in range(len(price2.columns))]  
    # 通过重新排序列来创建新的 DataFrame  
    # 注意：这里我们实际上没有移动数据，只是改变了列的显示顺序  
    new_df = price2[new_columns]  
    # 将新的 DataFrame 添加到列表中  
    new_dataframes2.append(new_df) 
    
new_dataframes2[0]
print(scalerClpr.inverse_transform([[-0.2702703]]))

In [ ]:
#数据集
import numpy as np
def split_data1(stock, lookback):
    data_raw = stock.to_numpy() 
    #print('data_raw.shape = ',data_raw.shape)
    data = []
    
    #print(len(data_raw))
    # you can free play（seq_length）
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    #print(data)
    
    data = np.array(data);
    #print('data.shape = ',data.shape)
    
    x_train = data[:,:-1,:]
    #print(x_train)
    y_train = data[:,-1,0:1]
    
    
    return [x_train, y_train]

In [ ]:
lookback = 20
xnow=[]
ynow=[]
x_now1, y_now1  = split_data1(price2, lookback)
xnow.append(x_now1)
ynow.append(y_now1)
for i in range(9):
    xt, yt = split_data1(new_dataframes2[i], lookback)
    xnow.append(xt)
    ynow.append(yt)

# print(xnow[0].shape)
# print(len(xnow[0]))
# print(xnow[0])

In [ ]:
import torch
import torch.nn as nn

x_now=[]
y_now_lstm=[]
y_now_gru=[]


for i in range(10):
    x_trainn = torch.from_numpy(xnow[i]).type(torch.Tensor)
    y_train_lstmn = torch.from_numpy(ynow[i]).type(torch.Tensor)
    y_train_grun = torch.from_numpy(ynow[i]).type(torch.Tensor)
    x_now.append(x_trainn)
    y_now_lstm.append(y_train_lstmn)
    y_now_gru.append(y_train_grun)
    
print(x_now[1].shape)

In [ ]:
import torch 
from sklearn.preprocessing import MinMaxScaler

pdtime = 240
pdclpr= []

price1 = data2[['Clpr', 'Trdvol', 'Trdsum', 'Dret', 'PE', 'PB', 'PCF', 'PS', 'EPS', 'ROE']]


for i in range (pdtime):
    Clpr = model0(x_now[0])
    Trdvol = model1(x_now[1])
    Trdsum = model2(x_now[2])
    Dret = model3(x_now[3])
    PE = model4(x_now[4])
    PB = model5(x_now[5])
    PCF = model6(x_now[6])
    PS = model7(x_now[7])
    EPS = model8(x_now[8])
    ROE = model9(x_now[9])
    print('before:',Clpr[0][0])
    Clpr1 = scalerClpr.inverse_transform(Clpr.detach().numpy())
    print('after:',Clpr1[0][0])
    # print("really:",scalerClpr.inverse_transform([[0.1998]]))
    # Trdvol = scalerTrdvol.inverse_transform(Trdvol.detach().numpy())
    # Trdsum = scalerTrdsum.inverse_transform(Trdsum.detach().numpy())
    # Dret = scalerDret.inverse_transform(Dret.detach().numpy())
    # PE = scalerPE.inverse_transform(PE.detach().numpy())
    # PB = scalerPB.inverse_transform(PB.detach().numpy())
    # PCF = scalerPCF.inverse_transform(PCF.detach().numpy())
    # PS = scalerPS.inverse_transform(PS.detach().numpy())
    # EPS = scalerEPS.inverse_transform(EPS.detach().numpy())
    # ROE = scalerROE.inverse_transform(ROE.detach().numpy())
    pdclpr.append(Clpr1[0][0])
    price2 = price2.drop(price2.index[0])
    print(Clpr[0][0])
    new_row_data = {  
    'Clpr': Clpr[0][0],  
    'Trdvol': Trdvol[0][0],  
    'Trdsum': Trdsum[0][0],  
    'Dret': Dret[0][0],  
    'PE': PE[0][0],  
    'PB': PB[0][0],  
    'PCF': PCF[0][0],  
    'PS': PS[0][0],  
    'EPS': EPS[0][0],  
    'ROE': ROE[0][0]  
    }  
    # 假设 new_row_data 是一个字典，包含要添加的新行的数据  
    new_row = pd.DataFrame([new_row_data]) 
    # print(new_row) 

    # 使用 concat 方法添加新行  
    price2 = pd.concat([price2, new_row], ignore_index=True)
    
    # price2= price1.copy()

    # scaler = MinMaxScaler(feature_range=(-1, 1))
    # scalerClpr = MinMaxScaler(feature_range=(-1, 1))
    # scalerTrdvol = MinMaxScaler(feature_range=(-1, 1))
    # scalerTrdsum = MinMaxScaler(feature_range=(-1, 1))
    # scalerDret = MinMaxScaler(feature_range=(-1, 1))
    # scalerPE = MinMaxScaler(feature_range=(-1, 1))
    # scalerPB = MinMaxScaler(feature_range=(-1, 1))
    # scalerPCF = MinMaxScaler(feature_range=(-1, 1))
    # scalerPS = MinMaxScaler(feature_range=(-1, 1))
    # scalerEPS = MinMaxScaler(feature_range=(-1, 1))
    # scalerROE = MinMaxScaler(feature_range=(-1, 1))
    # price2['Clpr'] = scalerClpr.fit_transform(price2['Clpr'].values.reshape(-1,1))
    # price2['Trdvol'] = scalerTrdvol.fit_transform(price2['Trdvol'].values.reshape(-1,1))
    # price2['Trdsum'] = scalerTrdsum.fit_transform(price2['Trdsum'].values.reshape(-1,1))
    # price2['Dret'] = scalerDret.fit_transform(price2['Dret'].values.reshape(-1,1))
    # price2['PE'] = scalerPE.fit_transform(price2['PE'].values.reshape(-1,1))
    # price2['PB'] = scalerPB.fit_transform(price2['PB'].values.reshape(-1,1))
    # price2['PCF'] = scalerPCF.fit_transform(price2['PCF'].values.reshape(-1,1))
    # price2['PS'] = scalerPS.fit_transform(price2['PS'].values.reshape(-1,1))
    # price2['EPS'] = scalerEPS.fit_transform(price2['EPS'].values.reshape(-1,1))
    # price2['ROE'] = scalerROE.fit_transform(price2['ROE'].values.reshape(-1,1))


    # 创建一个空的列表来存储新的 DataFrame 对象  
    new_dataframes2 = []  
    
    # 循环创建九个新的 DataFrame  
    for shift in range(9):  
        # 计算新的列顺序  
        new_columns = [price2.columns[(shift + 1 + idx) % len(price2.columns)] for idx in range(len(price2.columns))]  
        # 通过重新排序列来创建新的 DataFrame  
        # 注意：这里我们实际上没有移动数据，只是改变了列的显示顺序  
        new_df = price2[new_columns]  
        # 将新的 DataFrame 添加到列表中  
        new_dataframes2.append(new_df) 
        
    lookback = 20
    xnow=[]
    ynow=[]
    x_now1, y_now1  = split_data1(price2, lookback)
    xnow.append(x_now1)
    ynow.append(y_now1)
    for i in range(9):
        xt, yt = split_data1(new_dataframes2[i], lookback)
        xnow.append(xt)
        ynow.append(yt)
    
    x_now=[]
    y_now_lstm=[]
    y_now_gru=[]
    # print(xnow[0])

    for i in range(10):
        x_trainn = torch.from_numpy(xnow[i].astype(np.float64)).type(torch.Tensor)
        y_train_lstmn = torch.from_numpy(ynow[i].astype(np.float64)).type(torch.Tensor)
        y_train_grun = torch.from_numpy(ynow[i].astype(np.float64)).type(torch.Tensor)
        x_now.append(x_trainn)
        y_now_lstm.append(y_train_lstmn)
        y_now_gru.append(y_train_grun)
        
    # print(len(price2))



In [ ]:
#result
import matplotlib.pyplot as plt
import pandas as pd  

# 读取CSV文件，假设文件名为 'your_file.csv'  
df = pd.read_csv('RESSET_DRESSTK_2021__1really.csv')  

# 提取Clpr列并转换为列表  
clpr_list = df['Clpr'].tolist() 

x1 = range(len(clpr_list))

x2 = range(len(pdclpr))

# 创建一个新的图形  
plt.figure()  

# 绘制第一个列表  
plt.plot(x1, clpr_list, label='real')  

# 绘制第二个列表  
plt.plot(x2, pdclpr, label='predict')  

# 添加图例  
plt.legend()  

# 添加标题和标签  
plt.title('My Plot with Two Lists')  
plt.xlabel('X Axis Label')  
plt.ylabel('Y Axis Label')  


# 显示图形  
plt.show()
